# Python Apprentice Lecture 6: Time Complexity and Big-O Notation

이번 단원에서는 알고리즘의 효율성을 평가하는 핵심 개념인 **시간복잡도(Time Complexity)**와 **빅O 표기법(Big-O Notation)**을 배운다.

같은 결과를 내는 두 코드가 있을 때, 어느 것이 더 효율적인지 객관적으로 판단할 수 있는 능력은 프로그래머에게 필수적이다.

## 학습 목표
1. 알고리즘 성능 측정의 필요성 이해
2. 시간복잡도의 개념과 중요성 파악
3. 빅O 표기법의 의미와 사용법 습득
4. 주요 복잡도 클래스 구별 (O(1), O(log n), O(n), O(n log n), O(n²), O(2ⁿ))
5. 코드를 보고 시간복잡도 분석하는 방법 습득
6. 실전 정렬 알고리즘의 시간복잡도 분석

## 0. '연산'의 정의

시간복잡도를 분석할 때 세는 **기본 연산(Primitive Operations)**:

### 기본 연산의 예시
1. **변수 할당**: `x = 5`, `arr[i] = 10`
2. **산술 연산**: `+`, `-`, `*`, `/`, `%` 등
3. **비교 연산**: `==`, `<`, `>`, `<=`, `>=`, `!=`
4. **배열 접근**: `arr[i]` (인덱스로 직접 접근)
5. **리턴 문**: `return x`

### 함수 호출은?
- **호출 자체의 오버헤드**: O(1) (스택 프레임 생성 등, 보통 무시)
- **중요한 것**: 함수 **내부에서 수행하는 연산**

**복잡도 곱셈 법칙**: `A번 반복` × `각 반복마다 B 연산` = **O(A × B)**

예시:
```python
for i in range(n):      # n번 반복
    func_call()         # 함수의 복잡도가 O(k)라면
# 전체: O(n × k)
```

- `func_call()`이 O(1)이면 전체는 **O(n)**
- `func_call()`이 O(n)이면 전체는 **O(n²)**

---

In [1]:
for i in range(10): # n
    for k in range(i):  # n/2
        print(k, end=' ')
    print()

# n * n = n ^2


0 
0 1 
0 1 2 
0 1 2 3 
0 1 2 3 4 
0 1 2 3 4 5 
0 1 2 3 4 5 6 
0 1 2 3 4 5 6 7 
0 1 2 3 4 5 6 7 8 


### 0.1. 변수에 값을 저장하는 경우

```python
# 예시 1: 단순 변수 할당
x = 10  # 1회 연산
```
- 할당 연산 1회 → **O(1)**

```python
# 예시 2: 배열 요소 할당
arr[5] = 100  # 1회 연산
```
- 배열 인덱스 접근 + 할당 → **O(1)**

```python
# 예시 3: 반복문에서 변수 할당
for i in range(n):
    pass
```

**정확한 연산 횟수**:
- `i`에 값 할당: n회
- 종료 조건 확인(`i < n` 등): n+1회
- 총 **≈ 2n+1회** 연산

**실무적 이해**:
- "n번 반복"이므로 → **O(n)**
- 정확한 상수(2, +1 등)는 Big-O에서 무시됨

**참고**: C 스타일 루프 `for(i=0; i<n; i++)`도 마찬가지
- 초기화 1회 + 증가 n회 + 비교 n+1회 = 2n+2회 → **O(n)**

---

### 0.2. 산술 연산의 경우

```python
# 예시 1: 단순 덧셈과 할당
total = total + i
```
- 덧셈 연산 1회 + 변수 할당 1회 = 2회 연산 → **O(1)**

```python
# 예시 2: 복합 표현식
result = 2 * n + 3
```
- 곱셈 1회 (`2 * n`)
- 덧셈 1회 (`... + 3`)
- 변수 할당 1회 (`result = ...`)
- 총 3회 연산 → **O(1)**

**중요**: 여기서 `3`은 **리터럴(literal) 상수**로, 그 자체가 "할당"되는 것이 아니라 덧셈 연산의 **피연산자(operand)**입니다.

```python
# 예시 3: 반복문 내 산술 연산
total = 0
for i in range(n):
    total = total + i
```
- 루프가 n번 반복
- 각 반복마다 덧셈 1회 + 할당 1회 = 2회
- 총 2n회 연산 → **O(n)**

---

## 1. 왜 시간복잡도가 필요한가?

### 동기부여: 같은 결과, 다른 성능

1부터 n까지의 합을 구하는 두 가지 방법을 비교해보자.

In [3]:
import time

# 방법 1: 반복문 사용
def sum_method1(n):
    total = 0       # 1번
    for i in range(1, n+1):     # 1번
        total += i      # 2번 * n번(for) = 2n번
    return total        # 1번
# 2n+3번 -> O(n)
# n=10 -> 23번
# n=100 -> 203번
# n=1000 -> 2003번

# 방법 2: 수학 공식 사용
def sum_method2(n):
    return n * (n + 1) // 2     # 3번
# 3번 -> O(1)

# 테스트
n = 10000000

# 방법 1 시간 측정
start = time.time()
result1 = sum_method1(n)
end = time.time()
print(f"방법 1 결과: {result1}")
print(f"방법 1 실행 시간: {end - start:.10f}초")

# 방법 2 시간 측정
start = time.time()
result2 = sum_method2(n)
end = time.time()
print(f"\n방법 2 결과: {result2}")
print(f"방법 2 실행 시간: {end - start:.10f}초")

try:
    print(f"\n속도 차이: 방법 1이 약 {((end - start) / (end - start)) * 1000:.0f}배 더 오래 걸림")
except:
    print("\n속도 차이: 방법 1이 측정이 의미 없을 만큼 오래 걸림")

방법 1 결과: 50000005000000
방법 1 실행 시간: 0.6955964565초

방법 2 결과: 50000005000000
방법 2 실행 시간: 0.0000569820초

속도 차이: 방법 1이 약 1000배 더 오래 걸림


### 핵심 질문

- 두 코드 모두 정답을 출력한다
- 하지만 실행 시간이 크게 다르다
- **왜 이런 차이가 발생할까?**
- **어떻게 객관적으로 비교할 수 있을까?**

## 2. 성능 측정의 어려움

### 왜 초(second)로 측정하지 않을까?

시간을 직접 측정하는 방식의 문제점:

In [4]:
# 같은 코드를 여러 번 실행해보자
def simple_loop(n):
    total = 0
    for i in range(n):
        total += i
    return total

n = 100000
times = []

for trial in range(5):
    start = time.time()
    simple_loop(n)
    end = time.time()
    elapsed = end - start
    times.append(elapsed)
    print(f"시도 {trial+1}: {elapsed:.6f}초")

print(f"\n평균: {sum(times)/len(times):.6f}초")
print(f"최소: {min(times):.6f}초")
print(f"최대: {max(times):.6f}초")
print(f"편차: {max(times) - min(times):.6f}초")

시도 1: 0.007380초
시도 2: 0.005975초
시도 3: 0.005780초
시도 4: 0.006713초
시도 5: 0.006248초

평균: 0.006419초
최소: 0.005780초
최대: 0.007380초
편차: 0.001601초


### 시간 측정의 문제점

1. **컴퓨터마다 속도가 다름**
   - 내 노트북: 1초
   - 슈퍼컴퓨터: 0.001초
   - 오래된 컴퓨터: 10초

2. **같은 코드도 실행할 때마다 시간이 다름**
   - CPU 스케줄링
   - 다른 프로그램의 영향
   - 캐시 상태

3. **입력 크기에 따라 달라짐**
   - n=100: 매우 빠름
   - n=1,000,000: 느림

### 해결책: 연산 횟수로 측정

**"얼마나 빨리"가 아니라 "몇 번 연산하는가"를 센다!**

- 컴퓨터와 무관하게 객관적
- 입력 크기(n)에 따른 증가 패턴 파악 가능

## 3. 시간복잡도 개념 소개 (간단한 예시)

코드의 연산 횟수를 입력 크기 n의 함수로 표현한 것이 **시간복잡도**이다.

주요 복잡도 클래스를 간단한 예제로 살펴보자.

### O(1) - 상수 시간
**"입력 크기와 상관없이 항상 같은 시간"**

In [ ]:
# 배열 인덱스 접근
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(arr[0])  # n=10이든 n=1000이든 1번만 실행

# 변수 계산
result = 5 + 3  # 항상 1번

### O(log n) - 로그 시간
**"입력이 2배 → 연산은 +1번만 증가"**

In [ ]:
# 이진 탐색 개념: 매번 절반씩 줄어듦
# n=16 → 8 → 4 → 2 → 1 (4번)
# n=32 → 16 → 8 → 4 → 2 → 1 (5번)
# n이 2배가 되어도 연산은 1번만 증가!

def show_binary_search_steps(n):
    """이진 탐색의 단계 수 시각화"""
    steps = 0
    current = n
    print(f"n={n}일 때:")
    while current > 0:
        print(f"  단계 {steps+1}: {current}개")
        current //= 2
        steps += 1
    print(f"총 {steps}단계\n")
    return steps

show_binary_search_steps(16)
show_binary_search_steps(32)
show_binary_search_steps(1024)

### O(n) - 선형 시간
**"입력이 2배 → 연산도 2배"**

In [ ]:
# 배열 순회
def print_all(arr):
    for num in arr:  # n번 반복
        print(num) # 1번 출력 * n번

# n=5 → 5번
# n=10 → 10번
# n=100 → 100번

### O(n log n) - 선형로그 시간
**"n개를 log n 단계로 나눠서 처리"**

**핵심**: 절반씩 나누면 몇 단계? → **log₂(n) 단계**
- n=8: 8 → 4 → 2 → 1 (3단계, log₂8 = 3)
- n=16: 16 → 8 → 4 → 2 → 1 (4단계, log₂16 = 4)
- n=1000: 약 10단계 (log₂1000 ≈ 10)

**병합 정렬 예시**:
- 분할 단계: log n번 (절반씩 나눔)
- 각 단계마다: n개 원소를 병합
- **총: log n 단계 × n개 = n log n**

*(상세한 병합 정렬 코드는 Section 5.4에서 다룹니다)*

In [10]:
# 실제 이진 탐색 코드로 O(log n) 확인하기

def binary_search_steps(arr, target):
    """
    이진 탐색: 매 단계마다 탐색 범위가 절반으로 줄어듦
    """
    left = 0
    right = len(arr) - 1
    steps = 0

    print(f"배열 크기: {len(arr)}, 찾는 값: {target}")

    while left <= right:
        steps += 1
        mid = (left + right) // 2

        print(f"  단계 {steps}: 범위 [{left}~{right}], 중간={mid}, 값={arr[mid]}")

        if arr[mid] == target:
            print(f"  → 찾음! 총 {steps}번\n")
            return mid, steps
        elif arr[mid] < target:
            left = mid + 1  # 오른쪽 절반만 탐색
        else:
            right = mid - 1  # 왼쪽 절반만 탐색

    print(f"  → 없음. 총 {steps}번\n")
    return -1, steps

# 테스트
import math

# n=16 (log₂16 = 4)
arr = list(range(16))
_, steps = binary_search_steps(arr, 10)
print(f"예상: log₂(16) = {math.ceil(math.log2(16))}단계, 실제: {steps}단계\n")

# n=100 (log₂100 ≈ 7)
arr = list(range(100))
_, steps = binary_search_steps(arr, 73)
print(f"예상: log₂(100) ≈ {math.ceil(math.log2(100))}단계, 실제: {steps}단계\n")

# n=1000 (log₂1000 ≈ 10)
arr = list(range(1000))
_, steps = binary_search_steps(arr, 777)
print(f"예상: log₂(1000) ≈ {math.ceil(math.log2(1000))}단계, 실제: {steps}단계\n")

print("🔑 핵심: 매 단계마다 탐색 범위가 절반 → 총 log₂(n) 단계!")

배열 크기: 16, 찾는 값: 10
  단계 1: 범위 [0~15], 중간=7, 값=7
  단계 2: 범위 [8~15], 중간=11, 값=11
  단계 3: 범위 [8~10], 중간=9, 값=9
  단계 4: 범위 [10~10], 중간=10, 값=10
  → 찾음! 총 4번

예상: log₂(16) = 4단계, 실제: 4단계

배열 크기: 100, 찾는 값: 73
  단계 1: 범위 [0~99], 중간=49, 값=49
  단계 2: 범위 [50~99], 중간=74, 값=74
  단계 3: 범위 [50~73], 중간=61, 값=61
  단계 4: 범위 [62~73], 중간=67, 값=67
  단계 5: 범위 [68~73], 중간=70, 값=70
  단계 6: 범위 [71~73], 중간=72, 값=72
  단계 7: 범위 [73~73], 중간=73, 값=73
  → 찾음! 총 7번

예상: log₂(100) ≈ 7단계, 실제: 7단계

배열 크기: 1000, 찾는 값: 777
  단계 1: 범위 [0~999], 중간=499, 값=499
  단계 2: 범위 [500~999], 중간=749, 값=749
  단계 3: 범위 [750~999], 중간=874, 값=874
  단계 4: 범위 [750~873], 중간=811, 값=811
  단계 5: 범위 [750~810], 중간=780, 값=780
  단계 6: 범위 [750~779], 중간=764, 값=764
  단계 7: 범위 [765~779], 중간=772, 값=772
  단계 8: 범위 [773~779], 중간=776, 값=776
  단계 9: 범위 [777~779], 중간=778, 값=778
  단계 10: 범위 [777~777], 중간=777, 값=777
  → 찾음! 총 10번

예상: log₂(1000) ≈ 10단계, 실제: 10단계

🔑 핵심: 매 단계마다 탐색 범위가 절반 → 총 log₂(n) 단계!


### O(n²) - 이차 시간
**"입력이 2배 → 연산은 4배"**

In [ ]:
# 중첩 반복문
def print_pairs(arr):
    for i in range(len(arr)):      # n번
        for j in range(len(arr)):  # 각각 n번
            print(f"({arr[i]}, {arr[j]})")  # 총 n × n = n²번

def trap(arr):
    for i in range(len(arr)):
        for j in range(1000):  # O(1)
            pass
# n=10 → 100번
# n=100 → 10,000번

### O(2ⁿ) - 지수 시간
**"입력이 +1 → 연산은 2배"**

In [11]:
# 재귀 피보나치: 매번 2개로 갈라짐
call_count = 0

def fibonacci(n):
    global call_count
    call_count += 1

    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

for n in [5, 10, 15, 20]:
    call_count = 0
    result = fibonacci(n)
    print(f"fibonacci({n}) = {result}, 호출 횟수: {call_count}")

fibonacci(5) = 5, 호출 횟수: 15
fibonacci(10) = 55, 호출 횟수: 177
fibonacci(15) = 610, 호출 횟수: 1973
fibonacci(20) = 6765, 호출 횟수: 21891


### 복잡도 성장 비교

In [5]:
import math

# 다양한 n 값에 대한 복잡도 비교
print("n\t\t\tO(1)\tO(log n)\tO(n)\tO(n log n)\tO(n²)\t\tO(2ⁿ)")
print("="*80)

for n in [1, 10, 100]:
    o_1 = 1
    o_log_n = int(math.log2(n)) if n > 0 else 0
    o_n = n
    o_n_log_n = n * o_log_n
    o_n2 = n * n
    o_2n = "매우 큼" if n > 20 else 2**n
    
    print(f"{n}\t\t\t{o_1}\t\t{o_log_n}\t\t\t{o_n}\t\t{o_n_log_n}\t\t\t{o_n2:,}\t\t\t{o_2n}")

n			O(1)	O(log n)	O(n)	O(n log n)	O(n²)		O(2ⁿ)
1			1		0			1		0			1			2
10			1		3			10		30			100			1024
100			1		6			100		600			10,000			매우 큼


## 4. 빅O 표기법 도입

### 왜 빅O가 필요한가?

앞서 본 `sum_method1`의 정확한 연산 횟수를 세어보자:

In [24]:
def sum_with_count(n):
    ops = 0
    total = 0  # 1번 대입
    ops += 1
    
    for i in range(1, n+1):  # n번 반복
        total += i  # n번 덧셈 + n번 대입
        ops += 2
    
    # return total  # 1번
    ops += 1
    
    return total, ops

for n in [5, 10, 100]:
    result, ops = sum_with_count(n)
    print(f"n={n}: 약 {ops}번 연산 (정확히는 1 + 2n + 1 = {1 + 2*n + 1})")

n=5: 약 12번 연산 (정확히는 1 + 2n + 1 = 12)
n=10: 약 22번 연산 (정확히는 1 + 2n + 1 = 22)
n=100: 약 202번 연산 (정확히는 1 + 2n + 1 = 202)


### 빅O의 핵심 아이디어

**"정확한 횟수보다 증가 추세가 중요하다!"**

- `2n + 2`든 `5n + 10`이든 핵심은 **"n에 비례"**
- n이 충분히 크면 상수는 무시해도 됨
- 이를 **O(n)**으로 표기

### 빅O 표기법의 정의

**O(f(n))**: 입력 크기 n이 충분히 클 때, 최악의 경우 f(n)에 비례하는 시간이 걸린다

- O(1): 상수 시간
- O(log n): 로그 시간
- O(n): 선형 시간
- O(n log n): 선형로그 시간
- O(n²): 이차 시간
- O(2ⁿ): 지수 시간

## 5. 구체적 연산 횟수 세기 (상세 예제)

이제 각 복잡도별로 구체적인 예제를 통해 연산 횟수를 직접 세어보자.

### 5.1. O(1) - 상수 시간

#### 예제 1-1: 배열 특정 위치 접근

In [6]:
def get_first_and_last(arr):
    """
    배열의 첫 번째와 마지막 원소의 합을 반환
    """
    first = arr[0]           # ← 1번 (인덱스 접근 + 대입)
    last = arr[len(arr)-1]   # ← 1번 (인덱스 접근 + 대입)
    return first + last      # ← 1번 (덧셈)
    # 총 3번 (n과 무관!) o(1)

# 테스트
print("n=10일 때:", get_first_and_last(list(range(10))))
print("n=1000일 때:", get_first_and_last(list(range(1000))))
print("n=1,000,000일 때:", get_first_and_last(list(range(1000000))))
print("\n모든 경우 약 3번의 연산! → O(1)")

n=10일 때: 9
n=1000일 때: 999
n=1,000,000일 때: 999999

모든 경우 약 3번의 연산! → O(1)


**🔑 O(1) 포인트**:
- 인덱스로 직접 접근 (`arr[0]`, `arr[n-1]`)
- 반복문 없음
- n이 바뀌어도 연산 횟수 동일

In [13]:
a = {
    '철수': 1,
    '영희': 2,
    '길동': 3
}


True

#### 예제 1-2: 딕셔너리 조회

In [14]:
def get_student_score(scores_dict, name):
    """
    학생 이름으로 점수 조회
    """
    if name in scores_dict:  # ← 1번 (해시테이블 조회)
        return scores_dict[name]  # ← 1번 (해시테이블 접근)
    return -1  # ← 1번

# 테스트
scores = {f"Student{i}": i*10 for i in range(10000)}

import time
start = time.time()
result = get_student_score(scores, "Student5000")
end = time.time()

print(f"결과: {result}")
print(f"10,000명 중에서 찾는데 걸린 시간: {end-start:.8f}초")
print("딕셔너리 크기와 무관하게 거의 일정! → O(1)")

결과: 50000
10,000명 중에서 찾는데 걸린 시간: 0.00005770초
딕셔너리 크기와 무관하게 거의 일정! → O(1)


**🔑 O(1) 포인트**:
- 딕셔너리/해시테이블 조회는 평균 O(1)
- 모든 키를 순회하지 않음

### 5.2. O(log n) - 로그 시간

#### 예제 2-1: 이진 탐색

In [ ]:
def binary_search(arr, target):
    """
    정렬된 배열에서 이진 탐색
    """
    left, right = 0, len(arr) - 1  # 1번
    iterations = 0
    
    while left <= right:  # ← log n번 반복
        iterations += 1
        mid = (left + right) // 2  # 1번
        
        print(f"  반복 {iterations}: 범위 [{left}, {right}], 중간={mid}, 값={arr[mid]}")
        
        if arr[mid] == target:
            print(f"  찾음! 총 {iterations}번 반복")
            return mid
        elif arr[mid] < target:
            left = mid + 1  # ← 절반 버림!
        else:
            right = mid - 1  # ← 절반 버림!
    
    print(f"  못 찾음. 총 {iterations}번 반복")
    return -1

# 테스트
arr = list(range(0, 100, 2))  # [0, 2, 4, ..., 98]
print("배열 크기: 50")
print("50을 찾기:")
binary_search(arr, 50)

print("\n=" * 50)
arr = list(range(0, 2000, 2))  # [0, 2, 4, ..., 1998]
print("배열 크기: 1000 (20배 증가)")
print("1000을 찾기:")
binary_search(arr, 1000)

print("\nn이 20배 증가해도 반복 횟수는 조금만 증가! → O(log n)")

**🔑 O(log n) 포인트**:
- **매 반복마다 탐색 범위가 절반으로 줄어듦** (`left = mid + 1` 또는 `right = mid - 1`)
- 전체를 다 보지 않고 절반씩 제거
- "절반씩" = log₂n

#### 예제 2-2: 2의 거듭제곱 찾기

In [15]:
def find_power_of_two(n):
    """
    n 이상의 최소 2의 거듭제곱 찾기
    """
    power = 1  # 1번
    count = 0  # 1번
    
    print(f"n={n}일 때:")
    while power < n:  # ← log n번 반복
        count += 1
        power *= 2  # ← 2배씩 증가!
        print(f"  단계 {count}: power = {power}")
    
    print(f"총 {count}번 반복\n")
    return count

# 테스트
find_power_of_two(100)
find_power_of_two(1000)
find_power_of_two(1000000)

print("n이 10배씩 증가해도 반복은 조금씩만! → O(log n)")

n=100일 때:
  단계 1: power = 2
  단계 2: power = 4
  단계 3: power = 8
  단계 4: power = 16
  단계 5: power = 32
  단계 6: power = 64
  단계 7: power = 128
총 7번 반복

n=1000일 때:
  단계 1: power = 2
  단계 2: power = 4
  단계 3: power = 8
  단계 4: power = 16
  단계 5: power = 32
  단계 6: power = 64
  단계 7: power = 128
  단계 8: power = 256
  단계 9: power = 512
  단계 10: power = 1024
총 10번 반복

n=1000000일 때:
  단계 1: power = 2
  단계 2: power = 4
  단계 3: power = 8
  단계 4: power = 16
  단계 5: power = 32
  단계 6: power = 64
  단계 7: power = 128
  단계 8: power = 256
  단계 9: power = 512
  단계 10: power = 1024
  단계 11: power = 2048
  단계 12: power = 4096
  단계 13: power = 8192
  단계 14: power = 16384
  단계 15: power = 32768
  단계 16: power = 65536
  단계 17: power = 131072
  단계 18: power = 262144
  단계 19: power = 524288
  단계 20: power = 1048576
총 20번 반복

n이 10배씩 증가해도 반복은 조금씩만! → O(log n)


**🔑 O(log n) 포인트**:
- **변수가 2배(또는 절반)씩 변함** (`power *= 2`)
- 1 → 2 → 4 → 8 → ... 이런 식으로 기하급수적 증가/감소
- 1에서 n까지 2배씩 증가하면 log₂n번

### 5.3. O(n) - 선형 시간

#### 예제 3-1: 배열 최댓값 찾기

In [ ]:
def find_max(arr):
    """
    배열에서 최댓값 찾기
    """
    max_val = arr[0]  # 1번
    comparisons = 0
    
    for i in range(1, len(arr)):  # ← n-1번 반복
        comparisons += 1
        if arr[i] > max_val:  # n-1번 비교
            max_val = arr[i]  # 최악 n-1번
    
    return max_val, comparisons

# 테스트
for size in [10, 100, 1000]:
    arr = list(range(size))
    max_val, comparisons = find_max(arr)
    print(f"n={size:4d}: {comparisons}번 비교 (n-1과 거의 같음)")

print("\n비교 횟수가 n에 비례! → O(n)")

**🔑 O(n) 포인트**:
- **반복문이 배열 전체를 한 번 순회** (`for i in range(len(arr))`)
- 모든 원소를 다 봐야 함
- 입력 크기에 비례

#### 예제 3-2: 리스트 합계

In [ ]:
def array_sum_with_count(arr):
    """
    배열 합계와 연산 횟수 반환
    """
    total = 0  # 1번
    operations = 1
    
    for num in arr:  # ← n번 반복
        total += num  # n번 (덧셈 + 대입)
        operations += 1
    
    operations += 1  # return
    return total, operations

# 테스트
for size in [5, 100, 1000]:
    arr = list(range(1, size+1))
    total, ops = array_sum_with_count(arr)
    print(f"n={size:4d}: 약 {ops}번 연산 (1 + n + 1)")

print("\n연산 횟수 = 1 + n + 1 → 상수 무시 → O(n)")

**🔑 O(n) 포인트**:
- **단일 반복문**이 n번 실행
- 중첩 없음

### 5.4. O(n log n) - 선형로그 시간

#### 예제 4-1: 병합 정렬

In [16]:
merge_operations = 0

def merge_sort(arr, depth=0):
    """
    병합 정렬
    """
    global merge_operations
    
    if len(arr) <= 1:
        return arr
    
    # 분할 단계 ← log n 단계
    mid = len(arr) // 2
    indent = "  " * depth
    print(f"{indent}분할: {arr} → {arr[:mid]} | {arr[mid:]}")
    
    left = merge_sort(arr[:mid], depth+1)
    right = merge_sort(arr[mid:], depth+1)
    
    # 병합 단계 ← 각 단계마다 O(n)
    result = merge(left, right)
    print(f"{indent}병합: {left} + {right} → {result}")
    
    return result

def merge(left, right):
    """
    두 정렬된 배열을 병합
    """
    global merge_operations
    result = []
    i = j = 0
    
    # 모든 원소 비교하며 병합 ← O(n)
    while i < len(left) and j < len(right):
        merge_operations += 1
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    result.extend(left[i:])
    result.extend(right[j:])
    return result

# 테스트
arr = [38, 27, 43, 3, 9, 82, 10]
print(f"원본 배열 (n={len(arr)}): {arr}\n")
merge_operations = 0
sorted_arr = merge_sort(arr.copy())
print(f"\n정렬 결과: {sorted_arr}")
print(f"병합 연산 횟수: {merge_operations}")
print(f"예상 복잡도: n × log n = {len(arr)} × {math.ceil(math.log2(len(arr)))} ≈ {len(arr) * math.ceil(math.log2(len(arr)))}")

원본 배열 (n=7): [38, 27, 43, 3, 9, 82, 10]

분할: [38, 27, 43, 3, 9, 82, 10] → [38, 27, 43] | [3, 9, 82, 10]
  분할: [38, 27, 43] → [38] | [27, 43]
    분할: [27, 43] → [27] | [43]
    병합: [27] + [43] → [27, 43]
  병합: [38] + [27, 43] → [27, 38, 43]
  분할: [3, 9, 82, 10] → [3, 9] | [82, 10]
    분할: [3, 9] → [3] | [9]
    병합: [3] + [9] → [3, 9]
    분할: [82, 10] → [82] | [10]
    병합: [82] + [10] → [10, 82]
  병합: [3, 9] + [10, 82] → [3, 9, 10, 82]
병합: [27, 38, 43] + [3, 9, 10, 82] → [3, 9, 10, 27, 38, 43, 82]

정렬 결과: [3, 9, 10, 27, 38, 43, 82]
병합 연산 횟수: 13
예상 복잡도: n × log n = 7 × 3 ≈ 21


**🔑 O(n log n) 포인트**:
- **재귀적으로 절반씩 분할** (O(log n)) + **각 단계마다 전체 병합** (O(n))
- 분할 깊이: log n
- 각 깊이에서 처리: n
- 곱셈: n × log n

#### 예제 4-2: 각 원소마다 이진 탐색

In [ ]:
def binary_search_simple(arr, target):
    """간단한 이진 탐색 (반복 횟수 세지 않음)"""
    left, right = 0, len(arr) - 1
    iterations = 0
    
    while left <= right:
        iterations += 1
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid, iterations
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1, iterations

def search_all(sorted_arr, targets):
    """
    여러 target을 각각 이진 탐색
    """
    results = []
    total_iterations = 0
    
    for target in targets:  # ← n번
        # 각 target에 대해 이진 탐색 ← log n번
        index, iterations = binary_search_simple(sorted_arr, target)
        total_iterations += iterations
        results.append((target, index))
    
    return results, total_iterations

# 테스트
sorted_arr = list(range(0, 1000, 2))  # [0, 2, 4, ..., 998]
targets = [100, 200, 300, 400, 500]  # 5개 target

results, total_ops = search_all(sorted_arr, targets)
print(f"정렬된 배열 크기: {len(sorted_arr)}")
print(f"검색할 target 개수: {len(targets)}")
print(f"총 비교 횟수: {total_ops}")
print(f"예상: n × log n = {len(targets)} × {math.ceil(math.log2(len(sorted_arr)))} ≈ {len(targets) * math.ceil(math.log2(len(sorted_arr)))}")
print("\nn개 원소에 대해 각각 log n 작업 → O(n log n)")

**🔑 O(n log n) 포인트**:
- **O(n) 반복문** 안에서 **O(log n) 작업** 수행
- 중첩 구조: n × log n

### 5.5. O(n²) - 이차 시간

#### 예제 5-1: 중복 찾기

In [ ]:
def find_duplicates(arr):
    """
    배열에서 중복된 값 찾기 (비효율적 방법)
    """
    duplicates = []
    comparisons = 0
    
    for i in range(len(arr)):  # ← n번
        for j in range(i + 1, len(arr)):  # ← n번
            comparisons += 1
            if arr[i] == arr[j]:  # 각 쌍마다 1번
                if arr[i] not in duplicates:
                    duplicates.append(arr[i])
    
    return duplicates, comparisons

# 테스트
for size in [5, 10, 20]:
    arr = list(range(size))
    _, comparisons = find_duplicates(arr)
    expected = size * (size - 1) // 2
    print(f"n={size:2d}: {comparisons:3d}번 비교 (예상: {expected} = n(n-1)/2)")

print("\n비교 횟수 = n(n-1)/2 ≈ n²/2 → O(n²)")

**🔑 O(n²) 포인트**:
- **중첩된 반복문** (`for` 안에 `for`)
- 외부 n번 × 내부 n번 = n²
- `j = i + 1`이라도 여전히 O(n²) (상수 차이일 뿐)

#### 예제 5-2: 버블 정렬

In [17]:
def bubble_sort(arr):
    """
    버블 정렬
    """
    n = len(arr)
    arr = arr.copy()  # 원본 보존
    comparisons = 0
    swaps = 0
    
    for i in range(n):  # ← ?번
        for j in range(n - i - 1):  # ← ?번
            comparisons += 1
            if arr[j] > arr[j + 1]:
                # 교환
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swaps += 1
    
    return arr, comparisons, swaps

# 테스트
for size in [5, 10, 20]:
    arr = list(range(size, 0, -1))  # 역순 (최악의 경우)
    _, comparisons, swaps = bubble_sort(arr)
    expected = size * (size - 1) // 2
    print(f"n={size:2d}: {comparisons:3d}번 비교 (예상: {expected})")

print("\n이중 중첩 반복문 → O(n²)")

n= 5:  10번 비교 (예상: 10)
n=10:  45번 비교 (예상: 45)
n=20: 190번 비교 (예상: 190)

이중 중첩 반복문 → O(n²)


**🔑 O(n²) 포인트**:
- **이중 중첩 반복문**
- 내부 반복 횟수가 줄어들어도 (n-i-1) 여전히 O(n²)
- n + (n-1) + (n-2) + ... + 1 = n(n+1)/2 ≈ n²/2 → O(n²)

### 5.6. O(2ⁿ) - 지수 시간

#### 예제 6-1: 피보나치 (순수 재귀)

In [ ]:
fib_calls = 0

def fibonacci(n, depth=0):
    """
    피보나치 수 계산 (순수 재귀, 비효율적)
    """
    global fib_calls
    fib_calls += 1
    
    if n <= 1:
        return n
    
    # 두 번의 재귀 호출! ← 트리가 2갈래로 갈라짐
    return fibonacci(n - 1, depth+1) + fibonacci(n - 2, depth+1)

# 테스트
print("n\t결과\t호출 횟수\t2ⁿ")
print("="*50)
for n in [5, 10, 15, 20]:
    fib_calls = 0
    result = fibonacci(n)
    print(f"{n}\t{result}\t{fib_calls}\t\t{2**n}")

print("\n호출 횟수가 지수적으로 증가! → O(2ⁿ)")
print("주의: n=30 이상은 매우 오래 걸립니다!")

**🔑 O(2ⁿ) 포인트**:
- **재귀 호출이 2번** (트리가 두 갈래로 갈라짐)
- 각 단계마다 호출 수가 2배로 증가
- 깊이 n, 각 레벨에서 2의 거듭제곱 → 2ⁿ

#### 피보나치 호출 트리 시각화

In [27]:
def fibonacci_visualize(n, depth=0, prefix=""):
    """
    피보나치 호출 트리 시각화 (작은 n만 사용!)
    """
    print(f"{prefix}fib({n})")
    
    if n <= 1:
        return n
    
    # 왼쪽 가지
    left = fibonacci_visualize(n - 1, depth+1, prefix + "  ")
    # 오른쪽 가지
    right = fibonacci_visualize(n - 2, depth+1, prefix + "  ")
    
    return left + right

print("fibonacci(5)의 호출 트리:")
print("="*50)
fibonacci_visualize(5)
print("\n매번 2개로 갈라지는 것을 확인!")

fibonacci(5)의 호출 트리:
fib(5)
  fib(4)
    fib(3)
      fib(2)
        fib(1)
        fib(0)
      fib(1)
    fib(2)
      fib(1)
      fib(0)
  fib(3)
    fib(2)
      fib(1)
      fib(0)
    fib(1)

매번 2개로 갈라지는 것을 확인!


#### 예제 6-2: 모든 부분집합 생성

In [ ]:
def all_subsets(arr, index=0):
    """
    배열의 모든 부분집합 생성
    """
    if index == len(arr):
        return [[]]  # 빈 집합
    
    # 현재 원소 제외한 부분집합 ← 1번째 재귀
    without = all_subsets(arr, index + 1)
    
    # 현재 원소 포함한 부분집합 ← 2번째 재귀
    with_elem = []
    for subset in without:
        with_elem.append([arr[index]] + subset)
    
    return without + with_elem

# 테스트
for n in [3, 5, 8, 10]:
    arr = list(range(n))
    subsets = all_subsets(arr)
    print(f"n={n:2d}: 부분집합 개수 = {len(subsets):4d} (2ⁿ = {2**n})")

print("\n부분집합 개수 = 2ⁿ → O(2ⁿ)")

# 작은 예제로 부분집합 확인
print("\narr=[1,2,3]의 모든 부분집합:")
subsets = all_subsets([1, 2, 3])
for subset in subsets:
    print(subset)

**🔑 O(2ⁿ) 포인트**:
- 각 원소마다 **"포함" vs "제외" 2가지 선택**
- n개 원소 → 2ⁿ가지 조합
- 재귀가 2갈래로 분기

## 6. 빅O의 규칙

복잡도를 계산할 때 적용하는 주요 규칙들을 알아보자.

### 규칙 1: 상수 무시

**상수 계수는 무시한다**

In [ ]:
# 예제 1: 3n
def example1(arr):
    # 3번의 독립적인 순회
    for x in arr:  # n번
        print(x)
    
    for x in arr:  # n번
        print(x * 2)
    
    for x in arr:  # n번
        print(x * 3)
    
    # 총: 3n번
    # 하지만 빅O는 O(n)!

print("3n + 5 → O(n)")
print("100n → O(n)")
print("n/2 → O(n)")
print("\n이유: n이 충분히 크면 상수는 무의미")

### 규칙 2: 최고차항만 남김

**가장 빠르게 증가하는 항만 고려한다**

In [ ]:
# n² + 100n + 1000을 시각화
print("n\tn²\t100n\t1000\t합계\t\t지배항")
print("="*70)
for n in [10, 100, 1000, 10000]:
    n_squared = n * n
    hundred_n = 100 * n
    const = 1000
    total = n_squared + hundred_n + const
    print(f"{n}\t{n_squared:,}\t{hundred_n:,}\t{const}\t{total:,}\t\tn² ({n_squared/total*100:.1f}%)")

print("\nn이 커질수록 n²이 압도적!")
print("n² + 100n + 1000 → O(n²)")

**주요 변환 예시:**
- `n² + n` → O(n²)
- `n³ + n² + n` → O(n³)
- `2ⁿ + n²` → O(2ⁿ)
- `n log n + n` → O(n log n)

### 규칙 3: 최악의 경우 (Worst Case) 분석

**일반적으로 최악의 경우를 기준으로 한다**

In [ ]:
def linear_search(arr, target):
    """
    선형 탐색
    """
    for i in range(len(arr)):
        if arr[i] == target:
            return i
    return -1

arr = list(range(100))

# 최선의 경우 (Best Case): 첫 번째에 있음
print("최선: 0을 찾기 -", linear_search(arr, 0), "위치 (1번만 비교)")

# 평균의 경우 (Average Case): 중간쯤
print("평균: 50을 찾기 -", linear_search(arr, 50), "위치 (약 n/2번 비교)")

# 최악의 경우 (Worst Case): 마지막 또는 없음
print("최악: 99를 찾기 -", linear_search(arr, 99), "위치 (n번 비교)")
print("최악: 없는 값 -", linear_search(arr, 999), "(n번 비교)")

print("\n빅O는 최악의 경우를 기준으로 → O(n)")

## 7. 실전 분석 연습

코드를 보고 복잡도를 직접 분석해보자.

### 연습 1: 단일 반복문

In [ ]:
def exercise1(n):
    for i in range(n):
        print(i)

# O(n)

### 연습 2: 중첩 반복문

In [ ]:
def exercise2(n):
    for i in range(n):
        for j in range(n):
            print(i, j)

# 질문: 시간복잡도는?
# 답: O(n²)
# 이유: 이중 중첩 반복문, n × n

### 연습 3: 상수 반복 (함정!)

In [ ]:
def exercise3(n):
    for i in range(n):
        for j in range(100):  # 상수!
            print(i, j)

# 질문: 시간복잡도는?
# 답: O(n)
# 이유: n × 100 = 100n → 상수 무시 → O(n)

### 연습 4: 연속된 반복문

In [ ]:
def exercise4(n):
    # 첫 번째 루프
    for i in range(n):
        print(i)
    
    # 두 번째 루프
    for i in range(n):
        print(i)

# 질문: 시간복잡도는?
# 답: O(n)
# 이유: n + n = 2n → 상수 무시 → O(n)

### 연습 5: 다른 크기의 중첩

In [ ]:
def exercise5(n):
    for i in range(n):
        for j in range(i):  # i번 반복 (0, 1, 2, ..., n-1)
            print(i, j)

# 질문: 시간복잡도는?
# 답: O(n²)
# 이유: 0 + 1 + 2 + ... + (n-1) = n(n-1)/2 ≈ n²/2 → O(n²)

### 연습 6: 절반씩 줄이기

In [ ]:
def exercise6(n):
    i = n
    while i > 1:
        print(i)
        i = i // 2  # 절반씩 줄어듦

# 질문: 시간복잡도는?
# 답: O(log n)
# 이유: 매번 절반씩 줄어듦

### 연습 7: 재귀

In [ ]:
def exercise7(n):
    if n <= 1:
        return
    exercise7(n // 2)
    exercise7(n // 2)

# 질문: 시간복잡도는?
# 답: O(n)
# 이유: 매번 2개로 갈라지고(2), 깊이는 log n → 2^(log n) = n

**🔍 상세 분석**:

이 함수는 **재귀를 2번 호출**하지만 **O(2^n)이 아니라 O(n)**입니다!

**재귀 트리 구조** (n=16 예시):
```
레벨 0:              16                    ← 1개 노드
                    /  \
레벨 1:           8      8                 ← 2개 노드
                 / \    / \
레벨 2:         4   4  4   4               ← 4개 노드
               /|\ /|\/|\ /|\
레벨 3:       2 2 2 2 2 2 2 2             ← 8개 노드
레벨 4:      1 1 1 1 1 1 1 1 ...         ← 16개 노드
```

**핵심 관찰**:
- 트리 깊이: **log₂(n)** (n을 절반씩 나누면 log n번)
- 각 레벨의 노드 수: 1, 2, 4, 8, ..., n
- 마지막 레벨: **2^(log₂(n)) = n개**

**왜 2^(log₂(n)) = n 인가?**
- log₂(16) = 4 의미: "2를 4번 곱하면 16"
- 따라서 2^4 = 16
- 일반화: **2^(log₂(n)) = n**

**총 노드 수**:
```
1 + 2 + 4 + 8 + ... + n = 2n - 1 ≈ O(n)
```

**피보나치와 비교**:
- 피보나치 `fib(n-1) + fib(n-2)`: n이 **조금씩** 줄어듦 → 깊이 n → O(2^n)
- exercise7 `f(n//2) + f(n//2)`: n이 **절반씩** 줄어듦 → 깊이 log n → O(n)

### 연습 8: 복합 예제

In [ ]:
def exercise8(n):
    # Part 1: O(n)
    for i in range(n):
        print(i)
    
    # Part 2: O(n²)
    for i in range(n):
        for j in range(n):
            print(i, j)
    
    # Part 3: O(n)
    for i in range(n):
        print(i)

# 질문: 시간복잡도는?
# 답: O(n²)
# 이유: O(n) + O(n²) + O(n) = O(n²) (최고차항만 남김)

## 8. 정렬 알고리즘 복습과 시간복잡도 분석

이전 강의에서 배운 정렬 알고리즘들의 시간복잡도를 분석해보자.

### 8.1. 선택 정렬 (Selection Sort)

**동작 원리**:
- 배열에서 최솟값을 찾아 맨 앞과 교환
- 나머지 부분에서 최솟값을 찾아 두 번째 위치와 교환
- 이를 반복하여 정렬

In [21]:
def selection_sort(arr):
    """
    선택 정렬
    """
    arr = arr.copy()  # 원본 보존
    n = len(arr)
    comparisons = 0
    swaps = 0

    for i in range(n - 1):  # ← n-1번
        # 최솟값 찾기
        min_idx = i
        for j in range(i + 1, n):  # ← (n-1), (n-2), ..., 1번
            comparisons += 1
            if arr[j] < arr[min_idx]:
                min_idx = j

        # 교환
        if min_idx != i:
            arr[i], arr[min_idx] = arr[min_idx], arr[i]
            swaps += 1

    return arr, comparisons, swaps

# 테스트
test_arr = [64, 34, 25, 12, 22, 11, 90]
print("원본 배열:", test_arr)
sorted_arr, comparisons, swaps = selection_sort(test_arr)
print("정렬 결과:", sorted_arr)
print(f"비교 횟수: {comparisons}")
print(f"교환 횟수: {swaps}")
print(f"예상: (n-1) + (n-2) + ... + 1 = n(n-1)/2 = {len(test_arr)*(len(test_arr)-1)//2}")

원본 배열: [64, 34, 25, 12, 22, 11, 90]
정렬 결과: [11, 12, 22, 25, 34, 64, 90]
비교 횟수: 21
교환 횟수: 4
예상: (n-1) + (n-2) + ... + 1 = n(n-1)/2 = 21


**시간복잡도 분석**:

```python
for i in range(n - 1):          # n-1번
    for j in range(i + 1, n):   # (n-1), (n-2), ..., 1번
        # 비교 연산
```

- 비교 횟수: (n-1) + (n-2) + (n-3) + ... + 1
- 등차수열의 합: **n(n-1)/2 ≈ n²/2**
- 상수 무시: **O(n²)**

**특징**:
- 최선, 평균, 최악 모두 **O(n²)**
- 이미 정렬되어 있어도 모든 원소를 비교해야 함
- 교환 횟수가 적음 (최대 n-1번)
- 불안정 정렬 (Unstable Sort)

### 8.2. 버블 정렬 (Bubble Sort)

**동작 원리**:
- 인접한 두 원소를 비교하여 큰 값을 뒤로 보낸다
- 한 번의 패스(pass)로 가장 큰 값이 맨 뒤로 이동한다
- 나머지 부분에 대해 같은 과정을 반복한다

In [22]:
def bubble_sort(arr):
    """
    버블 정렬
    """
    arr = arr.copy()  # 원본 보존
    n = len(arr)
    comparisons = 0
    swaps = 0
    
    for i in range(n):  # ← n번
        for j in range(n - i - 1):  # ← (n-1), (n-2), ..., 1번
            comparisons += 1
            if arr[j] > arr[j + 1]:
                # 교환 (거품처럼 큰 값이 위로!)
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swaps += 1
    
    return arr, comparisons, swaps

# 테스트
test_arr = [64, 34, 25, 12, 22, 11, 90]
print("원본 배열:", test_arr)
sorted_arr, comparisons, swaps = bubble_sort(test_arr)
print("정렬 결과:", sorted_arr)
print(f"비교 횟수: {comparisons}")
print(f"교환 횟수: {swaps}")
print(f"예상: (n-1) + (n-2) + ... + 1 = n(n-1)/2 = {len(test_arr)*(len(test_arr)-1)//2}")

원본 배열: [64, 34, 25, 12, 22, 11, 90]
정렬 결과: [11, 12, 22, 25, 34, 64, 90]
비교 횟수: 21
교환 횟수: 14
예상: (n-1) + (n-2) + ... + 1 = n(n-1)/2 = 21


**시간복잡도 분석**:

```python
for i in range(n):              # n번
    for j in range(n - i - 1):  # (n-1), (n-2), ..., 1번
        # 비교 및 교환
```

- 비교 횟수: (n-1) + (n-2) + (n-3) + ... + 1
- 등차수열의 합: **n(n-1)/2 ≈ n²/2**
- 상수 무시: **O(n²)**

**특징**:
- 최악/평균: **O(n²)**
- 최선 (이미 정렬됨): O(n) - 최적화 버전에서만
- 안정 정렬 (Stable Sort)

### 8.3. 삽입 정렬 (Insertion Sort)

**동작 원리**:
- 배열의 앞부분은 이미 정렬되어 있다고 가정
- 다음 원소를 정렬된 부분의 적절한 위치에 삽입
- 모든 원소가 처리될 때까지 반복

In [23]:
def insertion_sort(arr):
    """
    삽입 정렬
    """
    arr = arr.copy()  # 원본 보존
    n = len(arr)
    comparisons = 0
    shifts = 0
    
    for i in range(1, n):  # ← n-1번
        key = arr[i]
        j = i - 1
        
        # 적절한 위치 찾기 (최악: i번)
        while j >= 0 and arr[j] > key:
            comparisons += 1
            arr[j + 1] = arr[j]
            shifts += 1
            j -= 1
        
        if j >= 0:
            comparisons += 1  # 마지막 비교
        
        arr[j + 1] = key
    
    return arr, comparisons, shifts

# 테스트
test_arr = [12, 11, 13, 5, 6]
print("원본 배열:", test_arr)
sorted_arr, comparisons, shifts = insertion_sort(test_arr)
print("정렬 결과:", sorted_arr)
print(f"비교 횟수: {comparisons}")
print(f"이동 횟수: {shifts}")

원본 배열: [12, 11, 13, 5, 6]
정렬 결과: [5, 6, 11, 12, 13]
비교 횟수: 9
이동 횟수: 7


**시간복잡도 분석**:

```python
for i in range(1, n):           # n-1번
    while j >= 0 and arr[j] > key:  # 최악: i번 (1, 2, 3, ..., n-1)
        # 이동 연산
```

- **최악의 경우** (역순 정렬):
  - 비교 및 이동: 1 + 2 + 3 + ... + (n-1) = **n(n-1)/2 ≈ O(n²)**

- **최선의 경우** (이미 정렬됨):
  - 각 원소당 1번씩만 비교 → **O(n)**

- **평균**: **O(n²)**

**특징**:
- 거의 정렬된 배열에서는 매우 효율적 (O(n)에 가까움)
- 안정 정렬

### 8.4. 퀵 정렬 (Quick Sort)

**동작 원리**:
- 피벗(pivot)을 선택한다
- 피벗보다 작은 값들을 왼쪽, 큰 값들을 오른쪽으로 분할
- 왼쪽과 오른쪽을 재귀적으로 정렬
- 분할 정복(Divide and Conquer) 알고리즘

In [24]:
def quick_sort(arr):
    """
    퀵 정렬
    """
    if len(arr) <= 1:
        return arr
    
    # 피벗 선택 (중간 값)
    pivot = arr[len(arr) // 2]
    
    # 분할 (각 O(n))
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    
    # 재귀적으로 정렬
    return quick_sort(left) + middle + quick_sort(right)

# 테스트
test_arr = [3, 6, 8, 10, 1, 2, 1]
print("원본 배열:", test_arr)
sorted_arr = quick_sort(test_arr)
print("정렬 결과:", sorted_arr)

원본 배열: [3, 6, 8, 10, 1, 2, 1]
정렬 결과: [1, 1, 2, 3, 6, 8, 10]


**시간복잡도 분석**:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    
    pivot = arr[len(arr) // 2]
    
    # 분할: O(n)
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    
    # 재귀: 왼쪽 + 오른쪽
    return quick_sort(left) + middle + quick_sort(right)
```

- **평균/최선의 경우**:
  - 피벗이 중간값에 가까움 → 균등하게 분할
  - 재귀 깊이: **log n**
  - 각 단계에서 분할: **O(n)**
  - 총: **O(n log n)**

- **최악의 경우** (이미 정렬됨, 피벗이 항상 최소/최대):
  - 한쪽으로만 분할 → 재귀 깊이 n
  - 총: **O(n²)**

**특징**:
- 평균적으로 가장 빠른 정렬 알고리즘 중 하나
- 피벗 선택이 중요 (랜덤 피벗으로 최악 회피 가능)
- 불안정 정렬

### 8.5. 병합 정렬 (Merge Sort)

**동작 원리**:
- 배열을 절반으로 나눈다 (분할)
- 각 절반을 재귀적으로 정렬한다
- 정렬된 두 배열을 병합한다
- 분할 정복(Divide and Conquer) 알고리즘

In [25]:
def merge_sort_with_count(arr):
    """
    병합 정렬 (비교 횟수 추적)
    """
    if len(arr) <= 1:
        return arr, 0

    # 분할
    mid = len(arr) // 2
    left, left_comps = merge_sort_with_count(arr[:mid])
    right, right_comps = merge_sort_with_count(arr[mid:])

    # 병합
    merged, merge_comps = merge(left, right)

    total_comps = left_comps + right_comps + merge_comps
    return merged, total_comps

def merge(left, right):
    """
    두 정렬된 배열을 병합
    """
    result = []
    i = j = 0
    comparisons = 0

    # 양쪽을 비교하며 병합
    while i < len(left) and j < len(right):
        comparisons += 1
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # 나머지 추가
    result.extend(left[i:])
    result.extend(right[j:])

    return result, comparisons

# 테스트
test_arr = [64, 34, 25, 12, 22, 11, 90]
print("원본 배열:", test_arr)
sorted_arr, comparisons = merge_sort_with_count(test_arr)
print("정렬 결과:", sorted_arr)
print(f"비교 횟수: {comparisons}")

import math
n = len(test_arr)
expected = n * math.ceil(math.log2(n))
print(f"예상: n × log₂(n) ≈ {n} × {math.ceil(math.log2(n))} ≈ {expected}")

원본 배열: [64, 34, 25, 12, 22, 11, 90]
정렬 결과: [11, 12, 22, 25, 34, 64, 90]
비교 횟수: 14
예상: n × log₂(n) ≈ 7 × 3 ≈ 21


**시간복잡도 분석**:

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = merge_sort(arr[:mid])    # ← 재귀적 분할
    right = merge_sort(arr[mid:])   # ← 재귀적 분할

    return merge(left, right)       # ← 병합 O(n)
```

- **분할 단계**: 배열을 절반씩 나눔 → 깊이 **log₂(n)**
- **병합 단계**: 각 단계에서 n개 원소를 병합 → **O(n)**
- **전체**: log₂(n) 단계 × 각 단계 O(n) = **O(n log n)**

**특징**:
- 최선, 평균, 최악 모두 **O(n log n)** (항상 일정!)
- 안정 정렬 (Stable Sort)
- 추가 메모리 필요 (공간복잡도 O(n))
- 큰 데이터셋에서 매우 효율적
- 재귀 호출 오버헤드 존재

### 8.6. 정렬 알고리즘 시간복잡도 비교

In [26]:
import time
import random

# 다양한 크기로 실험
sizes = [10, 50, 100, 500]

print("정렬 알고리즘 성능 비교 (랜덤 배열)\n")
print(f"{'n':<6} {'선택':<10} {'버블':<10} {'삽입':<10} {'퀵':<10}")
print("=" * 50)

for n in sizes:
    # 랜덤 배열 생성
    test_data = [random.randint(1, 1000) for _ in range(n)]
    
    times = []
    
    # 선택 정렬
    start = time.time()
    selection_sort(test_data.copy())
    times.append(time.time() - start)
    
    # 버블 정렬
    start = time.time()
    bubble_sort(test_data.copy())
    times.append(time.time() - start)
    
    # 삽입 정렬
    start = time.time()
    insertion_sort(test_data.copy())
    times.append(time.time() - start)
    
    # 퀵 정렬
    start = time.time()
    quick_sort(test_data.copy())
    times.append(time.time() - start)
    
    print(f"{n:<6} {times[0]:.6f}  {times[1]:.6f}  {times[2]:.6f}  {times[3]:.6f}")

print("\n→ n이 커질수록 O(n log n) 알고리즘이 훨씬 빠름!")

정렬 알고리즘 성능 비교 (랜덤 배열)

n      선택         버블         삽입         퀵         
10     0.000023  0.000018  0.000015  0.000020
50     0.000183  0.000257  0.000166  0.000087
100    0.000590  0.000989  0.000610  0.000226
500    0.016324  0.030021  0.015151  0.001398

→ n이 커질수록 O(n log n) 알고리즘이 훨씬 빠름!


### 정렬 알고리즘 요약표

| 알고리즘 | 최선 | 평균 | 최악 | 안정성 | 특징 |
|---------|------|------|------|--------|------|
| **선택 정렬** | O(n²) | O(n²) | O(n²) | ✗ | 항상 n²번 비교 |
| **버블 정렬** | O(n) | O(n²) | O(n²) | ✓ | 거의 정렬된 경우 빠름 |
| **삽입 정렬** | O(n) | O(n²) | O(n²) | ✓ | 거의 정렬된 경우 매우 빠름 |
| **퀵 정렬** | O(n log n) | O(n log n) | O(n²) | ✗ | 평균적으로 가장 빠름 |
| **병합 정렬** | O(n log n) | O(n log n) | O(n log n) | ✓ | 항상 n log n |

**핵심 교훈**:
- O(n²) 정렬: 작은 배열(<100)에 적합
- O(n log n) 정렬: 큰 배열(>100)에서 필수
- n=1000일 때 차이:
  - O(n²): 약 1,000,000번 연산
  - O(n log n): 약 10,000번 연산 (100배 차이!)

## 🎯 학습 정리

### 배운 내용

1. **시간복잡도의 필요성**
   - 같은 결과도 효율성이 다를 수 있음
   - 객관적 성능 측정 방법 필요
   - 연산 횟수로 측정

2. **빅O 표기법**
   - 입력 크기에 따른 증가 추세 표현
   - 상수와 낮은 차수 항 무시
   - 최악의 경우 기준

3. **주요 복잡도 클래스**
   - O(1): 상수 - 배열 접근, 딕셔너리 조회
   - O(log n): 로그 - 이진 탐색, 절반씩 줄이기
   - O(n): 선형 - 배열 순회
   - O(n log n): 선형로그 - 병합 정렬
   - O(n²): 이차 - 중첩 반복문
   - O(2ⁿ): 지수 - 재귀 피보나치, 부분집합

4. **복잡도 분석 포인트**
   - O(1): 반복 없음, 직접 접근
   - O(log n): 절반씩 줄어듦/증가
   - O(n): 단일 반복문
   - O(n log n): n번 × log n 작업
   - O(n²): 중첩 반복문
   - O(2ⁿ): 2갈래 재귀

5. **공간복잡도**
   - 메모리 사용량 측정
   - 시간 vs 공간 트레이드오프

### 복잡도 순서 (빠름 → 느림)

```
O(1) < O(log n) < O(n) < O(n log n) < O(n²) < O(2ⁿ) < O(n!)
```

### 실전 팁

1. **반복문 개수로 추정**
   - 반복문 없음 → O(1)
   - 1개 반복문 → O(n)
   - 2개 중첩 → O(n²)
   - 절반씩 → O(log n)

2. **최적화 우선순위**
   - O(2ⁿ) → O(n²): 필수!
   - O(n²) → O(n log n): 매우 중요
   - O(n log n) → O(n): 중요
   - O(n) → O(1): 상황에 따라

3. **주의사항**
   - 빅O는 충분히 큰 n일 때의 경향성
   - n이 작으면 상수가 중요할 수 있음
   - 실제 성능은 측정 필요

## 💡 추가 연습 문제

1. **복잡도 분석**: 다음 코드들의 시간복잡도를 분석하세요.
   ```python
   def mystery1(n):
       i = 1
       while i < n:
           i *= 3
   
   def mystery2(arr):
       for i in range(len(arr)):
           for j in range(i, len(arr)):
               print(arr[i], arr[j])
   ```

2. **실전 문제**: 정렬된 두 배열을 합쳐서 정렬된 하나의 배열을 만드는 함수를 O(n+m)으로 작성하세요.

3. **재귀 분석**: 하노이 탑의 시간복잡도는? (재귀 호출 패턴 분석)

이상으로 Python Apprentice Lecture 6을 마칩니다! 🚀

In [27]:
n = [1, 3, 6, 8, 9, 17]
m = [2, 6, 8, 10, 11, 12]

In [28]:
def merge(alist, blist):
    alen, blen = len(alist), len(blist)
    i, j = 0, 0
    result = []

    while i < alen and j < blen:
        if alist[i] <= blist[j]:
            result.append(alist[i])
            i += 1

        else:
            result.append(blist[j])
            j += 1

    if i == alen:
        result.extend(blist[j:])
    else:
        result.extend(alist[i:])